<a href="https://colab.research.google.com/github/said-racha/Hybrid-recommender-system-based-on-deep-learning-and-sentiment-analysis/blob/main/Amazon_pretraitements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import ast

#Traitement sur fichier json

In [ ]:
#lecteur du fichier json
amazon_json= pd.read_json('/content/drive/MyDrive/Datasets/Amazon video games/Video_Games_5.json',lines=True)
amazon_json[:3]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1,Wrong key,1403913600,"06 28, 2014"


In [ ]:
#transformation a un fichier csv
nomDuCsvFile='/content/drive/MyDrive/fichiers Csv/amazon_complet.csv'
amazon_json.to_csv(nomDuCsvFile, sep='|', index=False) 
amazon_complet_csv=pd.read_csv(nomDuCsvFile, delimiter='|')
amazon_complet_csv[:3]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1,Wrong key,1403913600,"06 28, 2014"


#Traitement sur fichier csv

##Créer et arranger le dataset réduit

In [ ]:
amazon_complet=pd.read_csv('/content/drive/MyDrive/fichiers Csv/amazon_complet.csv', delimiter='|')
amazon_reduit=amazon_complet.iloc[:,[0,1,4,5]] #recupperer dans un nouveau dataset que les colonnes que nous souhaitons étudier
amazon_reduit[:3]

,reviewerID,asin,reviewText,overall
0,A2HD75EMZR8QLN,0700099867,Installing the game was a struggle (because of...,1
1,A3UR8NLLY1ZHCX,0700099867,If you like rally cars get this game you will ...,4
2,A1INA0F5CWW3J4,0700099867,1st shipment received a book instead of the ga...,1


In [ ]:
amazon_reduit.rename(columns = {'reviewerID':'user_id'}, inplace = True)
amazon_reduit.rename(columns = {'asin':'item_id'}, inplace = True)
amazon_reduit.rename(columns = {'reviewText':'text'}, inplace = True)
amazon_reduit.rename(columns = {'overall':'rating'}, inplace = True)
amazon_reduit[:3]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user_id,item_id,text,rating
0,A2HD75EMZR8QLN,0700099867,Installing the game was a struggle (because of...,1
1,A3UR8NLLY1ZHCX,0700099867,If you like rally cars get this game you will ...,4
2,A1INA0F5CWW3J4,0700099867,1st shipment received a book instead of the ga...,1


##Coder les ids

In [ ]:
amazon_ratings_columns = ['user_id','item_id','rating']
amazon_reviews_columns = ['user_id','item_id','text']
amazon_full_dataset_columns = ['user_id','item_id','text','rating']

amazon_sort_ids=amazon_reduit.copy()

#codifier les id
amazon_sort_ids.user_id = amazon_reduit.user_id.astype('category').cat.codes.values
amazon_sort_ids.item_id = amazon_reduit.item_id.astype('category').cat.codes.values

#Df ratings avec id encodé et organisés
amazon_ratings = amazon_sort_ids[amazon_ratings_columns]
amazon_ratings = amazon_ratings.drop_duplicates()
amazon_ratings = amazon_ratings.sort_values(by=['user_id','item_id'])


#Df reviews avec id encodé et organisés
amazon_reviews = amazon_sort_ids[amazon_reviews_columns]
amazon_reviews = amazon_reviews.drop_duplicates()
amazon_reviews = amazon_reviews.sort_values(by=['user_id','item_id'])


#Df total avec id encodé et organisés
amazon_full_dataset = amazon_sort_ids[amazon_full_dataset_columns]
amazon_full_dataset = amazon_full_dataset.drop_duplicates()
amazon_full_dataset = amazon_full_dataset.sort_values(by=['user_id','item_id'])


amazon_ratings.reset_index(drop=True,inplace = True)
amazon_reviews.reset_index(drop=True,inplace = True)
amazon_full_dataset.reset_index(drop=True,inplace = True)

amazon_ratings.to_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/not cleaned/amazon_ratings.csv', sep='|', index=False,header = True)
amazon_reviews.to_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/not cleaned/amazon_reviews.csv', sep='|', index=False,header = True)
amazon_full_dataset.to_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/not cleaned/amazon_full_dataset.csv', sep='|', index=False,header = True)


In [ ]:
amazon_ratings[:3]

,user_id,item_id,rating
0,0,9656,5
1,0,9658,5
2,0,9664,5


In [ ]:
amazon_reviews[:3]

,user_id,item_id,text
0,0,9656,A highly known skylander in all of the skyland...
1,0,9658,"Chill is a water skylander, which means she ca..."
2,0,9664,"Pop Fizz, an alchemist in potions is powerful...."


In [ ]:
amazon_full_dataset[:3]

,user_id,item_id,text,rating
0,0,9656,A highly known skylander in all of the skyland...,5
1,0,9658,"Chill is a water skylander, which means she ca...",5
2,0,9664,"Pop Fizz, an alchemist in potions is powerful....",5


##Cleaning text

In [ ]:
amazon_ratings=pd.read_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/not cleaned/amazon_ratings.csv', delimiter='|')
amazon_reviews=pd.read_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/not cleaned/amazon_reviews.csv', delimiter='|')
amazon_full_dataset=pd.read_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/not cleaned/amazon_full_dataset.csv', delimiter='|')

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
import nltk

from nltk.tokenize import word_tokenize
nltk.download('punkt')

from nltk.corpus import stopwords
nltk.download('stopwords')

"""
import gensim
from gensim.models import Word2Vec
import os"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'\nimport gensim\nfrom gensim.models import Word2Vec\nimport os'

In [ ]:
str_values = amazon_full_dataset.text.apply(lambda x: type(x) == str)
amazon_dataset=amazon_full_dataset[str_values]

amazon_dataset=amazon_dataset.iloc[:131248]
amazon_dataset.reset_index(drop=True,inplace = True)

In [ ]:
#fonction pour nettoyer le texte donné en entrée
def cleaning_text(text) :
  test_sentenceSansEsp = re.sub("\s+"," ", text)

  #Removing extra spaces with regexp
  appos = {
  "aren't" : "are not",
  "can't" : "cannot",
  "couldn't" : "could not",
  "didn't" : "did not",
  "doesn't" : "does not",
  "don't" : "do not",
  "hadn't" : "had not",
  "hasn't" : "has not",
  "haven't" : "have not",
  "he'd" : "he would",
  "he'll" : "he will",
  "he's" : "he is",
  "i'd" : "I would",
  "i'd" : "I had",
  "i'll" : "I will",
  "i'm" : "I am",
  "isn't" : "is not",
  "it's" : "it is",
  "it'll":"it will",
  "i've" : "I have",
  "let's" : "let us",
  "mightn't" : "might not",
  "mustn't" : "must not",
  "shan't" : "shall not",
  "she'd" : "she would",
  "she'll" : "she will",
  "she's" : "she is",
  "shouldn't" : "should not",
  "that's" : "that is",
  "there's" : "there is",
  "they'd" : "they would",
  "they'll" : "they will",
  "they're" : "they are",
  "they've" : "they have",
  "we'd" : "we would",
  "we're" : "we are",
  "weren't" : "were not",
  "we've" : "we have",
  "what'll" : "what will",
  "what're" : "what are",
  "what's" : "what is",
  "what've" : "what have",
  "where's" : "where is",
  "who'd" : "who would",
  "who'll" : "who will",
  "who're" : "who are",
  "who's" : "who is",
  "who've" : "who have",
  "won't" : "will not",
  "wouldn't" : "would not",
  "you'd" : "you would",
  "you'll" : "you will",
  "you're" : "you are",
  "you've" : "you have",
  "'re": " are",
  "wasn't": "was not",
  "we'll":"we will",
  "didn't": "did not"
  }
  words = test_sentenceSansEsp.split()
  test_sentenceReformed = [appos[word] if word in appos else word for word in words]
  test_sentenceReformed = " ".join(test_sentenceReformed) 

  #rendre le texte en minuscule
  test_sentenceMinuscule = test_sentenceReformed.lower()
  
  #tokenzing (transformer la phrase en liste de mots)
  test_sentenceTokens = word_tokenize(test_sentenceMinuscule)

  #Ne garder que les mots (sup les nombre, ponctuation ...)
  test_sentenceAlphab = [word for word in test_sentenceTokens if word.isalpha()]

  #personaliser sa liste de stop words a supprimer
  stop_words = stopwords.words('english')
  negWords=['no','nor','not']
  stop_words_sansNegation=[]

  for w in stop_words :
    if(w not in negWords):
      stop_words_sansNegation.append(w)

  #supprimer les stop words
  test_sentenceSansStopWords= [i for i in test_sentenceAlphab if i not in stop_words_sansNegation]
  text_cleaned=' '.join(word for word in test_sentenceSansStopWords)
  
  return text_cleaned
  


def cleaning_text_df(df_col) :
  emb=[]
  for text in tqdm(df_col) :
    emb.append(cleaning_text(text))
  
  return emb

In [ ]:
#nettoyer les reviews dans le dataset
for i in range(amazon_dataset.shape[0]):
  amazon_dataset.iloc[i,2]=cleaning_text(amazon_dataset.iloc[i,2])

amazon_dataset

,user_id,item_id,text,rating
0,0,9656,highly known skylander skylander fans trigger ...,5
1,0,9658,chill water skylander means swim water unless ...,5
2,0,9664,pop fizz alchemist potions powerful magic skyl...,5
3,0,9676,drobot new lightcore character tech not sold l...,1
4,0,9683,crusher strong sturdy slow even though rocking...,5
...,...,...,...,...
131243,13925,6745,simply not fit standard model gently used far ...,1
131244,13925,6985,used precision instrument gaming general compu...,5
131245,13926,687,loved size new pstwo still overpriced buy xbox...,1
131246,13926,1392,game bit short fun played first came still lik...,5


In [ ]:
amazon_ratings_ceaned=amazon_dataset[['user_id','item_id','rating']]
amazon_reviews_ceaned=amazon_dataset[['user_id','item_id','text']]

amazon_ratings_ceaned.to_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/cleaned/amazon_ratings_ceaned.csv',sep='|', index=False,header = True)
amazon_reviews_ceaned.to_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/cleaned/amazon_reviews_ceaned.csv',sep='|', index=False,header = True)
amazon_dataset.to_csv('/content/drive/MyDrive/fichiers Csv/Datasets amazon avec ids sorted/cleaned/amazon_dataset.csv',sep='|', index=False,header = True)
